In [7]:
from IPython import get_ipython
get_ipython().magic('reset -sf')
import chromedriver_autoinstaller
import chromedriver_binary
import os
import requests
from selenium.webdriver.common.keys import Keys
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import datetime 
from datetime import datetime
chromedriver_autoinstaller.install()
os.getcwd()

'C:\\Users\\clj585\\OneDrive - Northwestern University\\data_feeds\\zacks\\codes'

In [8]:
def setOptions():
    options = Options()
    options.add_experimental_option("prefs", {
       "download.default_directory":"C:\\Users\\clj585\\OneDrive - Northwestern University\\data_feeds\\zacks\\data\\webscrape", 
        #Change download dir
        "download.prompt_for_download": False,
        "download.directory_upgrade": True
    })
    return options

In [9]:
def login(driver):
    driver.get("https://www.zacks.com/")
    #popup = driver.find_element(By.XPATH, "/html/body/footer/div[1]/div[2]/div/div/div/div[1]")
    #popup.click()
    
    signInDropFown = driver.find_element(By.XPATH, '/html/body/header/ul/li[2]/a')
    signInDropFown.click()
    usernameField = driver.find_element(By.XPATH, '/html/body/header/ul/li[2]/div/form/div[1]/input')
    usernameField.send_keys('fundalytix@gmail.com') #add login info here
    passwordField = driver.find_element(By.XPATH, '/html/body/header/ul/li[2]/div/form/div[2]/input')
    passwordField.send_keys('Matt1980**') #add login info here
    loginButton = driver.find_element(By.XPATH, '/html/body/header/ul/li[2]/div/form/p[2]/input')
    loginButton.click()
    return driver

In [10]:
def getCompletedDownloads(driver):
    if not driver.current_url.startswith("chrome://downloads"):
        driver.get("chrome://downloads/")

    return driver.execute_script("""
        var items = document.querySelector('downloads-manager')
            .shadowRoot.getElementById('downloadsList').items;

        if (items.every(e => e.state === "COMPLETE"))
            return items.length;
        """)


In [11]:
def checkDownloads(downloadCount, driver):
    driver.execute_script("window.open();")
    driver.switch_to.window(driver.window_handles[1])

    downloadedCSV = getCompletedDownloads(driver)
    while (downloadedCSV != None) and (downloadedCSV < downloadCount):
        time.sleep(.5)
        downloadedCSV = getCompletedDownloads(driver)
    time.sleep(.5)


In [12]:
def main():
    csvsDownloaded = 0
    options = setOptions()
    driver = webdriver.Chrome(options=options) #Change exec path
    driver.set_window_size(1920, 1080)

    driver = login(driver)
    driver.get("https://www.zacks.com/screening/stock-screener")

    delay = 10 # seconds

    try:
        driver.switch_to.frame(driver.find_element(By.ID, 'screenerContent'))
        myScreenButton = driver.find_element(By.XPATH, 
                                "/html/body/main/section/div/div[1]/div/button[5]")
        myScreenButton.click()
        
        runCustomButton = WebDriverWait(driver, delay).until(
            EC.element_to_be_clickable((By.XPATH, 
            "/html/body/main/section/div/div[6]/section/div/div/div/div/table/tbody/tr[1]/td[3]/a[1]")))
        runCustomButton.click()
        
        downloadCSVButton = WebDriverWait(driver, delay).until(EC.element_to_be_clickable(
            (By.XPATH, "/html/body/main/section/div/div[4]/div/div[1]/div[2]/div[1]/div/div[1]/a[1]")))
        downloadCSVButton.click()
        
        csvsDownloaded += 1 #used to keep track of how many downloads have been processed
        checkDownloads(csvsDownloaded, driver)
        
        # If for whatever reason the above script fails after 2 tries, need to alert us 
        # i.e. there was a Christmas holiday popup that blocked logging in function to run 
        print("success")                     

        driver.execute_script("window.close();")

    finally:
        driver.quit()

if __name__ == "__main__":
    main()

success
